In [2]:
import sys; sys.path.append('methods/')
import os
from regridding import *

In [3]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [4]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def nearest_px(x,y,da):
   
    # get squared distance from (x,y) to each point
    dist2 = (da.lat - x)**2 + (da.lon - y)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 0.125)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["lat", "lon"])
   
    return val

In [5]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile for the 2019 constituencies
sf = geopandas.read_file("Westminster_Parliamentary_Constituencies_December_2019_Boundaries_UK_BUC")
sf

,pcon19cd,pcon19nm,bng_e,bng_n,long,lat,objectid,st_lengths,st_areasha,geometry
0,E14000530,Aldershot,484884,155126,-0.78410,51.288952,1,37241.703656,5.258835e+07,"POLYGON ((485406.902 159918.603, 486138.297 15..."
1,E14000531,Aldridge-Brownhills,404723,302568,-1.93166,52.620869,2,35258.924699,4.398713e+07,"POLYGON ((406519.098 305054.298, 405782.501 30..."
2,E14000532,Altrincham and Sale West,374132,389051,-2.39049,53.397659,3,40812.004306,5.087472e+07,"POLYGON ((379104.096 393143.903, 377127.904 39..."
3,E14000533,Amber Valley,440478,349675,-1.39770,53.042831,4,54421.854957,1.246901e+08,"POLYGON ((444868.402 353958.100, 444113.303 35..."
4,E14000534,Arundel and South Downs,510686,115542,-0.42635,50.928711,5,207778.021149,6.469497e+08,"POLYGON ((506641.497 128757.203, 506389.800 12..."
...,...,...,...,...,...,...,...,...,...,...
645,W07000076,Caerphilly,316860,188276,-3.20144,51.587181,646,63865.409578,1.150569e+08,"POLYGON ((315131.401 200637.498, 315308.299 19..."
646,W07000077,Islwyn,320517,196276,-3.15048,51.659618,647,59239.492373,1.117145e+08,"POLYGON ((322947.397 199706.204, 322974.804 19..."
647,W07000078,Vale of Glamorgan,302293,173081,-3.40742,51.448250,648,91342.706619,3.148419e+08,"POLYGON ((302260.803 179531.999, 302783.903 17..."
648,W07000079,Cardiff West,312437,178939,-3.26294,51.502571,649,36926.700752,5.081285e+07,"POLYGON ((312599.599 182852.798, 312692.900 18..."


In [7]:
filesnames=["CORDEX-nearterm_rcp45_conseq-dry-days.nc","CORDEX-nearterm_rcp45_pr_DJF.nc","CORDEX-nearterm_rcp45_pr_JJA.nc","CORDEX-nearterm_rcp45_rx5day.nc","CORDEX-nearterm_rcp45_temp_ANN.nc","CORDEX-nearterm_rcp45_txx.nc","ERA5-trend_conseq-dry-days.nc","ERA5-trend_pr_DJF.nc","ERA5-trend_pr_JJA.nc","ERA5-trend_rx5day.nc","ERA5-trend_temp_ANN.nc","ERA5-trend_txx.nc"]
varnames_in=["cdd_anom","pr_anom","pr_anom","rx5day_anom","t_anom","txx_anom","cdd_trend","pr_trend","pr_trend","rx5day_trend","t_trend","txx_trend"]
varnames_out=["cdd_anom","prdjf_anom","prjja_anom","rx5day_anom","t_anom","txx_anom","cdd_trend","prdjf_trend","prjja_trend","rx5day_trend","t_trend","txx_trend"]

In [8]:
for i in range(filesnames.__len__()):
    fnm = "../gridded_data/Copernicus_Atlas_data/"+filesnames[i]
    ds = xr.open_dataset(fnm)
    #ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
    da=ds[varnames_in[i]]
    #print(da)
    rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "objectid")
    #print(rm)
    # apply the regionmask to the data and average over the x & y dimensions
    region_da = da.where(rm).mean(["lat", "lon"])
    #print(region_da)
    # find nearest neighbour (this takes the most time)
    region_nn = xr.concat([nearest_px(sf.lat[i],sf.long[i],da).expand_dims(region = [sf.objectid[i]]).assign_coords(constituency = ("region", [sf.pcon19cd[i]])) for i in range(len(sf))], "region")
    print(region_nn)
    # combine regionmask with nearest neighbour where regionmask didn't pick anything up
    region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")
    region_all=region_all.assign_coords(name=('region',sf.pcon19nm))
    # output the values into a csv file
    region_all.to_dataframe(name=varnames_out[i]).reset_index().to_csv(varnames_out[i]+'_oldconstituencies.csv')
    print("outputting ",varnames_out[i]+'_oldconstituencies.csv')




<xarray.DataArray 'cdd_anom' (region: 650)>
array([ 2.42757797e-01, -1.60142899e-01, -4.22463417e-01,  7.24601746e-02,
        3.89131546e-01, -1.89130783e-01,  4.40580368e-01, -4.07248497e-01,
        5.34780502e-01,  4.48554993e-01,  2.86954880e-01, -4.42037582e-02,
       -4.42037582e-02, -2.44202614e-01,  2.15217590e-01,  4.90579605e-01,
       -2.99276352e-01,  5.13044357e-01, -9.78288651e-02,  3.63042831e-01,
        2.92755127e-01,  2.20289230e-01,  2.65939713e-01,  2.42031097e-01,
       -4.46377754e-01,  1.02178574e-01, -4.19562340e-01,  4.20289993e-01,
        3.21016312e-01, -4.18839455e-01,  4.41970825e-02, -2.73189545e-01,
        4.41970825e-02, -1.23233795e-02,  4.41970825e-02,  4.41970825e-02,
       -1.60142899e-01,  4.41970825e-02, -1.23233795e-02, -3.21743965e-01,
       -4.99999046e-01, -4.19565201e-01, -2.81162262e-01, -3.60873222e-01,
       -3.17390442e-01, -5.31884193e-01,  3.55073929e-01, -6.73904419e-02,
       -4.09420967e-01, -4.09420967e-01, -4.57245827e-01

In [9]:
# Combine into a big dataframe
import pandas
all_files = os.listdir()    
csv_files = list(filter(lambda f: f.endswith('_oldconstituencies.csv'), all_files))
for i in range(csv_files.__len__()):
    this_csv=pandas.read_csv(csv_files[i])
    if i == 0:
        csv_out=this_csv.iloc[:,[2,3,4]]
    else:
        segments=csv_files[i].split("_")
        var='_'.join(segments[:2])
        csv_out[var]=this_csv.iloc[:,4]

csv_out.to_csv('../by_mp/rcp45_climate_oldconstituencies.csv',index=False)




In [10]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def nearest_px(x,y,da):
   
    print('x:',x,' y:',y)
    # get squared distance from (x,y) to each point
    dist2 = (da.lat - x)**2 + (da.lon - y)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 0.125)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["lat", "lon"])
   
    return val

In [11]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile for the 2024 constituencies
sf = geopandas.read_file("PCON_MAY_2024_UK_BFE")
sf

,PCON24CD,PCON24NM,PCON24NMW,BNG_E,BNG_N,LAT,LONG,GlobalID,geometry
0,E14001063,Aldershot,None,484716,155270,51.2903,-0.78648,b8b6c57e-6fe4-432c-9130-f4d6e4598d19,"POLYGON Z ((483364.601 160961.805 0.000, 48337..."
1,E14001064,Aldridge-Brownhills,None,404720,301030,52.6071,-1.93173,d3c0545a-9c68-4095-9b85-330c7141dbf2,"POLYGON Z ((406519.098 305054.298 0.000, 40648..."
2,E14001065,Altrincham and Sale West,None,374132,389051,53.3977,-2.39049,a7e18d2c-8dd0-4f1f-b158-48799d7f6259,"POLYGON Z ((377443.302 393344.296 0.000, 37745..."
3,E14001066,Amber Valley,None,440478,349674,53.0428,-1.39771,e38d2edf-19ce-4090-ab8e-c78449d3983f,"POLYGON Z ((436223.299 356984.804 0.000, 43624..."
4,E14001067,Arundel and South Downs,None,497309,118530,50.9580,-0.61585,4a4d2a3a-a7a5-4a91-a90f-0692b459c37c,"POLYGON Z ((505688.454 133874.110 0.000, 50569..."
...,...,...,...,...,...,...,...,...,...
645,W07000108,Swansea West,Gorllewin Abertawe,264670,195124,51.6386,-3.95702,ab9c216c-374a-4dda-aabb-b5d4095808c5,"POLYGON Z ((266627.504 191650.700 0.000, 26662..."
646,W07000109,Torfaen,Torfaen,327459,200480,51.6984,-3.05102,fe1b6677-bcf4-49d8-a516-85b42b10af00,"POLYGON Z ((333723.000 192653.903 0.000, 33372..."
647,W07000110,Vale of Glamorgan,Bro Morgannwg,301298,173080,51.4481,-3.42174,43710abb-9c64-41bd-9f14-6e8120252dc2,"POLYGON Z ((302260.803 179531.999 0.000, 30226..."
648,W07000111,Wrexham,Wrecsam,337298,348629,53.0313,-2.93642,95c39b06-f367-4432-9aaa-de2a48654814,"POLYGON Z ((323631.902 349914.197 0.000, 32363..."


In [12]:
for i in range(filesnames.__len__()):
    fnm = "../gridded_data/Copernicus_Atlas_data/"+filesnames[i]
    ds = xr.open_dataset(fnm)
    da=ds[varnames_in[i]]
    rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False)
    # apply the regionmask to the data and average over the x & y dimensions
    region_da = da.where(rm).mean(["lat", "lon"])
    region_all=region_all.assign_coords(constituency=('region',sf.PCON24CD))
    # find nearest neighbour (this takes the most time)
    region_nn = xr.concat([nearest_px(sf.LAT[i],sf.LONG[i],da).expand_dims(region = [region_da.region[i]]).assign_coords(constituency = ("region", [sf.PCON24CD[i]])) for i in range(len(sf))], "region")
    print(region_nn)
    # combine regionmask with nearest neighbour where regionmask didn't pick anything up
    region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")
    region_all=region_all.assign_coords(name=('region',sf.PCON24NM))
    # output the values into a csv file
    region_all.to_dataframe(name=varnames_out[i]).reset_index().to_csv(varnames_out[i]+'_newconstituencies.csv')
    print("outputting ",varnames_out[i]+'_newconstituencies.csv')

x: 51.2903  y: -0.78648
x: 52.6071  y: -1.93173
x: 53.3977  y: -2.39049
x: 53.0428  y: -1.39771
x: 50.958  y: -0.61585
x: 53.104  y: -1.25411
x: 51.1434  y: 1.01829
x: 53.4859  y: -2.11217
x: 51.8399  y: -0.61999
x: 51.9908  y: -1.43463
x: 51.5393  y: 0.106043
x: 53.5761  y: -1.45061
x: 53.5174  y: -1.38372
x: 54.3567  y: -3.23122
x: 51.6057  y: 0.440099
x: 51.2481  y: -1.1558
x: 53.3606  y: -0.99444
x: 51.395  y: -2.36832
x: 51.4664  y: -0.15703
x: 51.5594  y: -0.58484
x: 51.4009  y: -0.0331
x: 52.1348  y: -0.45508
x: 51.4977  y: -0.07125
x: 51.5223  y: -0.05638
x: 53.7477  y: -0.12941
x: 50.9381  y: 0.525092
x: 51.4624  y: 0.164996
x: 51.8823  y: -1.24575
x: 53.3745  y: -3.04191
x: 52.4535  y: -1.95751
x: 52.532  y: -1.84562
x: 52.4405  y: -1.87636
x: 52.4888  y: -1.79707
x: 52.4815  y: -1.88993
x: 52.4168  y: -1.97366
x: 52.5246  y: -1.91267
x: 52.4249  y: -1.93195
x: 52.4665  y: -1.80729
x: 54.6608  y: -1.95045
x: 53.7473  y: -2.50048
x: 53.5247  y: -2.21169
x: 53.8896  y: -3.01311

In [13]:
all_files = os.listdir()    
csv_files = list(filter(lambda f: f.endswith('_newconstituencies.csv'), all_files))
for i in range(csv_files.__len__()):
    this_csv=pandas.read_csv(csv_files[i])
    if i == 0:
        csv_out=this_csv.iloc[:,[2,3,4]]
    else:
        segments=csv_files[i].split("_")
        var='_'.join(segments[:2])
        csv_out[var]=this_csv.iloc[:,4]

csv_out.to_csv('../by_mp/rcp45_climate_newconstituencies.csv',index=False)